<a href="https://colab.research.google.com/github/Fjoelsak/highway-env/blob/THM-workshop/THM-workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"> <img src="https://www.thm.de/_thm/logos/thm.svg" width="200"></p>

# Reinforcement learning Workshop 

## Unsere challenge: Ein automatisiertes Parksystem
Dazu betrachten wir das **parking-v0** Umgebungsmodell von [eleurent](https://github.com/eleurent/highway-env).
Es liegt eine zielkonditionierte kontinuierliche Regelaufgabe vor, bei der ein Agent ein **Auto fährt**, indem er das Gaspedal and den Lenkwinkel regelt und mit einer bestimmten Ausrichtung **in einer Parklücke einparken soll**.
Dazu betrachten wir den Use Case eines Einparkmanövers auf einer Freifläche mit eingezeichneten Linien der senkrechten Parklücken und keinen weiteren Verkehrsteilnehmern, wie sie beispielsweise auf einem Supermarktparkplatz vorliegen kann.

![alt text](https://github.com/Fjoelsak/highway-env/blob/gh-media/docs/media/parking-env.gif?raw=true)


Zunächst installieren wir das Umgebungsmodell und importieren einige packages zur Visualisierung.

In [1]:
# Install environment and visualization dependencies 
!pip install -q git+https://github.com/Fjoelsak/highway-env#egg=highway-env
!pip install -q gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -q -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Environment
import gym
import highway_env
import numpy as np

# Visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import trange
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor
import base64

# IO
from pathlib import Path

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Zur Visualisierung bauen wir uns eine kleine Helferfunktion

In [0]:
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("video").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 300px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

## Let's try it!

Zum Ausprobieren lädt man nun das Umgebungsmodell und führt einige willkürlich ausgewählten actions aus.

In [0]:
env = gym.make("parking-v0")
env = Monitor(env, './video', force=True, video_callable=lambda episode: True)
env.reset()
done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
env.close()
show_video()

Was passiert hier genau?

*   die Umgebung **initialisiert** und **reseten**. 
*   in der while-Schleife wird die jeweilige **action** des Agenten festgelegt und ein Zeitschritt (**step**) durchgeführt


das heißt, der Agent führt die Action im Umgebungsmodell aus, dass sich dementsprechend verändert. Zusätzlich wird ein reward-Signal ausgegeben. Done ist lediglich ein flag, ob das Ziel erreicht ist und info weitere Informationen zum Zeitschritt.

Im vorgegebenen Umgebungsmodell kann ein Agent das Gaspedal (die **Beschleunigung**) und den **Lenkwinkel** regeln. Diese sind jeweils auf bestimmte Werte begrenzt. Demnach ist eine `action = [beschleunigung, lenkwinkel]`

In [4]:
print("Action:", action)

Action: [-0.8996684  0.937322 ]


Das Umgebungsmodell ist ein `GoalEnv`, das heißt der Agent bekommt ein Dictionary, dass sowohl die derzeitige **Observation** als auch das Ziel erhält. Dabei besteht die derzeitige Observation aus 

*   `x, y` dem aktuellen Standort des Autos
*   `vx, vy`der aktuellen Geschwindigkeit des Autos
*   `cos_h, sin_h` der derzeitigen Ausrichtung des Autos





In [0]:
print("Observation:", obs)

Das **reward**-Signal ist letztlich wieder skalar und wird berechnet aus einer gewichteten p-Norm der Differenz zwischen derzeitigem Status ($goal_{\text{achieved}}$) und definiertem Ziel ($goal_{\text{desired}}$). Dabei wurde den einzelnen Elementen der Differenz beider Stati Gewichte ($w$) zugewiesen, damit das übergeordnete Ziel: mit richtiger Ausrichtung in der Parklücke eingeparkt, erreicht wird. Die Nähe zum Zielort wird dabei besonders hoch gewichtet.
<center>
$ r = -|(goal_{\text{desired}} - goal_{\text{achieved}})\cdot w|^{0.5}$
</center>

Dabei wird bewusst ein negativer **reward** berechnet, da das Ziel der RL Algorithmen eine Maximierung des reward Signals zum Ziel hat.

In [0]:
print("Reward:", reward)

## Eine mögliche Lösung



In [0]:
%tensorflow_version 1.x
!pip install -q stable-baselines==2.10.0
!pip install -q git+https://github.com/Fjoelsak/highway-env#egg=highway-env

# Environment
import gym
import highway_env

# Agent
from stable_baselines import HER, SAC

Training

In [0]:
env = gym.make("parking-ActionRepeat-v0")
model = HER('MlpPolicy', env, SAC, n_sampled_goal=4,
            goal_selection_strategy='future',
            verbose=1, buffer_size=int(1e6),
            learning_rate=1e-3,
            gamma=0.9, batch_size=256,
            policy_kwargs=dict(layers=[256, 256, 256]))
model.learn(int(2e4))

Test the policy

In [0]:
env = gym.make("parking-ActionRepeat-v0")
env = Monitor(env, './video', force=True, video_callable=lambda episode: True)
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    env.unwrapped.automatic_rendering_callback = env.video_recorder.capture_frame
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
env.close()
show_video()